In [17]:
# 파서 태그 -
PARSER_TAG = [
    r"{{{#!html ",
    r"{{{#!wiki .*?\n",
    r"{{{#!folding .*?\n",
    r"{{{(#[A-Za-z0-9,]) ",
    r"{{{([+\-][1-5]) ",
    r"\[\[",
    r"\[\*",
    r"\[([A-Za-z가-힣]+)"
]

#파서 태그 라인
PARSER_TAG_LINE = [
    r"'''''",
    r"'''",
    r"''",
    r"__",
    r"~~",
    r"--",
    r"\^\^",
    r",,"
]

# 반드시 맨 첫줄에 와야 하는 파서 태그
PARSER_TAG_CR = [
    r"\n\u0020+(\*|1\.|A\.|I\.|i\.)",
    r"\n>",
    r"\n\|[^\s]",
    r"\n##",
]

# macro value
PARSER_TAG_MACRO = [
    'html',
    'wiki',
    'folding',
    'color',
    'size',
    'link',
    'ref',
    'macro'
]

#
PARSER_TAG_LINE_MACRO = [
   'bold-italic',
   'bold',
   'italic',
   'underline',
   'strike',
   'strike2',
   'sup',
   'sub'
]

PARSER_TAG_CR_MACRO = [
    'list',
    'bq',
    'table',
    'comment'
]

In [18]:
# 매크로 맵 - 결과
# {"position": [[시작,끝], [시작,끝],... ], "value":[매크로1, 매크로2, ...]
def macro_map(text: str):

    forced_return = False  # 처음에 개행기호 집어넣기
    if text[0] != '\n':
        text = '\n'+text  # 문법 원활하게 처리하기 위해 텍스트 처음에 개행기호 집어넣기
        forced_return = True

    txt_length = len(text)  # 길이
    txt_lines = list(map(lambda x: x+'\n', text.split('\n')))  # 글을 출력하기
    txt_lines[-1] = txt_lines[-1][:-1]  # 마지막 \n 지우기
    txt_position = [len("".join(txt_lines[:k])) for k, val in enumerate(txt_lines)] # 글의 위치값
    res = {"position": [], "value": []}
    macro_stack = []
    i = int(forced_return)  # 낱자 번호
    frn = int(forced_return)  # 위치 정할 때 사용.
    ln = 0 # 줄 번호

    while i < txt_length:
        
        # 가로선 매크로
        if re.match(r"\n-{4,10}\n", text[i-1:]):
            pattern_end = re.match(r"(^|\n)-{4,10}\n", text[i-1:]).end()
            res['position'].append([i - frn, i + pattern_end -1 - frn])
            res['value'].append('hr')
            i += pattern_end -1

        #헤더 매크로
        elif re.match(r"\n=.*?=\s*\n", text[i-1:]):
            pattern_end = re.match(r"\n=.*?=\s*\n", text[i-1:]).end()
            res['position'].append([i - frn, i+pattern_end - 1 - frn])
            res['value'].append('header')
            i += pattern_end - 1
        
        # 주석 매크로
        elif re.match(r"\n##", text[i-1:]):
            pattern_end = text[i].find('\n')
            res['position'].append([i - frn, i + pattern_end - frn])
            res['value'].append('comment')
            i += pattern_end
        
        # nowiki/pre 매크로
        elif re.match(r"{{{[^#+\-]", text[i:]):
            # 줄에 있으면 
            if re.search(r"}}}[^}]", txt_lines[ln]):
                closed_pos = re.search(r"}}}[^}]", text[i:]).start()
                res['position'].append([i - frn, i + closed_pos + 3 -frn ])
                res['value'].append('nowiki')
                i += closed_pos + 3 -frn
            else:
                closed_pos = re.search(r"}}}", text[i:]).start()
                res['position'].append([i - frn, i + closed_pos + 3 -frn ])
                res['value'].append('pre')
                i += closed_pos + 3 -frn
        
        # syntax 매크로
        elif re.match(r"{{{#!(syntax|source)", text[i:]):
            closed_pos = re.search(r"}}}", text[i:]).start()
            res['position'].append([i - frn, i + closed_pos + 3 -frn ])
            res['value'].append('syntax')
            i += closed_pos + 3 -frn

        # 나머지
        else:
            # 줄의 처음에 시작하는 매크로 분석
            for cri, tag1 in enumerate(PARSER_TAG_CR):
                if re.match(tag1, text[i-1:]):
                    macro_stack.append([PARSER_TAG_CR_MACRO[cri], i - frn])
                    i += re.match(tag1, text[i-1:]).end()-1
                    break
            else:
                
                # linear_parser_tag 분석
                for pli, tag2 in enumerate(PARSER_TAG_LINE):
                    if re.match(tag2, text[i:]):
                        if len(macro_stack)>0 and macro_stack[-1][0] == PARSER_TAG_LINE_MACRO[pli]:
                            macro_val = macro_stack.pop()
                            res['position'].append([macro_val[1], i + 3 - frn])
                            res['value'].append(PARSER_TAG_LINE_MACRO[pli])
                            i += len(tag2)
                        else:
                            macro_stack.append([PARSER_TAG_LINE_MACRO[pli], i - frn])
                            i += len(tag2)
                        break

                else:

                    # 파서 태그 분석
                    for pi, tag3 in enumerate(PARSER_TAG):
                        if re.match(tag3, text[i:]):
                            macro_stack.append([PARSER_TAG_MACRO[pi], i - frn])
                            i += re.match(tag3, text[i:]).end()
                            break
                    else:
                        #닫히는 태그인지 확인해보자
                        if len(macro_stack) > 0:
                            cur_macro = macro_stack[-1][0]
                            MACRO_RENDER_PROCESSOR = ['html', 'wiki', 'folding', 'syntax', 'color', 'size']

                            if i <  txt_length -2 and cur_macro in MACRO_RENDER_PROCESSOR and text[i:i+3] == "}}}":

                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 3 - frn])
                                res['value'].append(macro_val[0])
                                i += 3
                            elif i< txt_length -1 and cur_macro == "link" and text[i:i+2]=="]]":

                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 2 - frn])
                                res['value'].append(macro_val[0])
                                i += 2
                            elif cur_macro in ['ref', 'macro'] and text[i] == "]":

                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 1 - frn])
                                res['value'].append(macro_val[0])
                                i += 1

                            elif cur_macro == 'table' and re.match(r"\|\|(\n[^|]|\n$|$)", text[i:]):
                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 3 - frn] if i< txt_length-3 else [macro_val[1], i + 2 - frn] )
                                res['value'].append(macro_val[0])
                                i += 3 if i< txt_length-3 else 2

                            elif cur_macro == 'list' and (re.match(r"($|\n$|\n{2})", text[i:]) or re.match(r"\n[^\u0020]", text[i:]) or re.match(r"\n\u0020+[^*1AIi]", text[i:])):
                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 1 - frn])
                                res['value'].append(macro_val[0])
                                i += 1

                            elif cur_macro == 'bq' and (i == txt_length-1 or (text[i] == '\n' and text[i+1] != ">")):
                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 1 - frn])
                                res['value'].append(macro_val[0])
                                i += 1

                            elif cur_macro == 'comment' and (i< txt_length -2 or (text[i] == '\n' and  text[i+1:i+3] != '##')):
                                macro_val = macro_stack.pop()
                                res['position'].append([macro_val[1], i + 1 - frn])
                                res['value'].append(macro_val[0])
                                i += 1

                            elif i>0 and text[i] == '\n' and text[i-1] != '\n' and text[i-1] != '\n' and set(map(lambda x: x[0], macro_stack)).isdisjoint({'table', 'list', 'bq', 'comment'}):
                                res['position'].append([i - frn, i + 1 -frn])
                                res['value'].append('br')
                                i +=1
                            else:
                                i +=1
                        else:
                            i += 1
        # ln값 재조정
        if ln< len(txt_position) and i > txt_position[ln]:
            ln = len(list(filter(lambda x: x < i, txt_position)))

    res['remain_stack'] = macro_stack
    return res

In [3]:
namumark_cont2 = """
[include(틀:한시적 넘겨주기, other1=대한민국 제20대 대통령 선거의 사전투표 진행 상황, rd1=제20대 대통령 선거, anchor1=사전 투표율)]
[목차]
== 개요 ==
{{{+1 事前投票 / Early ballot }}}

[[선거]] 당일 투표가 어려운 유권자가 별다른 신고 없이 본 선거일 이전에 [[투표]]하는 제도. [[미국]]에서 [[2000년]] 조기투표 제도로 도입하고 나서 [[일본]], [[대한민국]] 등에도 도입되었다.

== [[대한민국]] ==
[[파일:1332325_article_99_20150424142312.jpg]]

관외 사전투표를 하는 [[대한민국 육군]]의 [[병(군인)|병사]] 유권자

[[파일:442824_233424_055.jpg]]

관내 사전투표를 하는 한 [[장애인]] 유권자

[youtube(6xnzoKxQg_A)][[중앙선거관리위원회]] 제작 [[제20대 국회의원 선거]] 사전투표 [[광고]]

[youtube(1OGPe1NM6sw)][[중앙선거관리위원회]] 제작 [[제20대 국회의원 선거]] 사전투표 홍보영상 (출연자 [[에이핑크]])

=== 법령 ===
||'''[[공직선거법]]'''
'''제148조(사전투표소의 설치)'''
① 구·시·군[[선거관리위원회]]는 선거일 전 5일부터 2일 동안(이하 "사전투표기간"이라 한다) 관할구역(선거구가 해당 구·시·군의 관할구역보다 작은 경우에는 해당 선거구를 말한다)의 읍·면·동마다 1개소씩 사전투표소를 설치·운영하여야 한다. 다만, 읍·면·동 관할구역에 군부대 밀집지역 등이 있는 경우에는 해당 지역에 사전투표소를 추가로 설치·운영할 수 있다.
② 구·시·군선거관리위원회는 제1항에 따라 사전투표소를 설치할 때에는 선거일 전 9일까지 그 명칭·소재지 및 설치·운영기간을 공고하고, 선거사무장 또는 선거연락소장에게 이를 통지하여야 하며, 관할구역 안의 투표구마다 5개소에 공고문을 첩부하여야 한다. 사전투표소의 설치장소를 변경한 때에도 또한 같다.

'''제158조(사전투표)'''
① 선거인([[거소투표]]자와 [[선상투표]]자는 제외한다)은 누구든지 사전투표기간 중에 사전투표소에 가서 투표할 수 있다.||

=== 역사 ===
[[http://info.nec.go.kr/bizcommon/popup/popup_search_prevoteForm.xhtml?electionId=0020200415|20년4월15일선거사전투표소검색링크]]
[[2013년]]에 치러진 [[2013년 상반기 재보궐선거]]부터 도입된  제도로[* 전국단위 선거로는 [[2014년]]에 치러진 [[제6회 전국동시지방선거]]가 최초이다.], 기존 [[부재자 투표]] 제도를 대체하였다. 부재자 신고를 하지 않아도 되도록 통합선거인명부에 입각하여 전국 [[읍(행정구역)|읍]], [[면(행정구역)|면]], [[동(행정구역)|동]] 단위로 사전투표소를 설치, [[선거권]]을 행사하도록 하고 있다. 전국 단위의 선거 전산망을 이용하므로 본 투표와 마찬가지로 공인된 신분증[* 관공서 or 공공 기관이 발행하고 사진이 붙어 있는 신분증. 정식 등록된 사립학교의 학생증도 가능하다.]만 들고 가면 꼭 자기 동네가 아니라 전국 어디에서도 투표가 가능하다. 주의할 것은 단위가 읍, 면, 동이지 꼭 [[주민센터]]나 읍, 면사무소에 설치할 필요는 없다는 점이다. 절대 다수는 주민센터나 읍/면사무소이지만, 일부의 경우 구/시/군청, 문화센터, 자치회관, 학교 체육관, 철도역[* 예를 들어 2016년 실시된 20대 국회의원 선거의 경우 서울 용산구 남영동과 한강로동 사전투표소가 각각 서울역과 용산역 맞이방에 설치되어 문전성시를 이루었다. 그리고 부재자투표 시절부터 인천국제공항에는 꼬박꼬박 사전투표소가 설치되고 있다.] 등과 같은 다른 곳에 설치되는 경우도 있고, 심지어 두 곳이 끝에서 끝인 경우도 있어서 자기가 투표할 곳의 사전투표소가 어디 있는지 꼭 조회하고 가야 헛걸음하지 않을 수 있다.

도입 초기에는 투표율 1/5가량을 담당했으나, 점점 사전투표제에 대한 국민들의 인지도가 높아지면서 최근에 진행된 두 선거인 [[제19대 대통령 선거]]와 [[제7회 전국동시지방선거]]에서는 사전투표율이 총 투표율의 1/3 정도를 차지했다.

대한민국에서는 사전투표와 조기투표의 두 용어가 혼재되어 쓰이고 있었다가 공직선거법에 의해 이 제도가 공식적으로 도입되면서 공식 명칭은 사전투표로 정착되었다. 하지만 한국의 사전투표와 구별하기 위해 외국의 선거를 설명할 때는 그 이후에도 조기투표라는 단어를 사용하기도 한다.

=== 절차 ===
자기 주소지 읍면동의 투표소에 간 경우에는 관내선거인에 줄을 서야 되고, 다른 지역 투표소에 간 경우 관외선거인에 줄을 서야 한다. 단, 같은 시/군/구의 다른 읍면동에 간 경우에는 선거에 따라 관내선거인인지 관외선거인인지가 다르다.[* 같은 시/군/구이면서 자기 동네에서 투표할 후보자와 그 읍면동에서 투표하는 후보자가 완벽히 일치할 때만 관내선거인이 되고, 조금이라도 다르면 관외선거인이 된다. 즉 대통령 선거의 경우 관내선거인이 되지만, 지방선거의 경우에는 구/시/군 의원이 같은 지역구인 바로 옆동네인 경우에는 관내선거인, 다른 경우에는 관외선거인이 된다. 국회의원의 경우에는 갑을병으로 나뉜 곳이 있으므로 선거구에 따라 다르다.] 이는 투표 절차가 다르기 때문인데, 관내선거인의 경우 투표용지만 배부받아서 기표소에서 찍은 다음에 투표용지를 접고 나와서 그냥 투표함에 넣으면 끝이지만, 관외선거인의 경우 신분확인 후 투표용지와 회송용 봉투를 출력해 같이 주고, 투표한 이후 기표소 안에서 회송용 봉투에 투표 용지를 집어 넣고 봉한 다음에 나와서 봉투채로 투표함에 넣어야 끝난다.[* 대통령선거의 경우 어차피 전국이 똑같으니 투표용지도 같은텐데 왜 투표용지를 출력하냐 싶겠지만, 자세히 보면 사실 대통령선거도 지역별로 투표용지가 다르다. 투표용지 맨 위에 선관위 직인이 인쇄되어 있는데, 바로 이 직인이 각 구/시/군별로 다르고 또한 부정선거 방지를 위해서는 달라야 하기 때문에 대통령 선거라도 타 지역 사람이 사전투표소에 오면 해당 유권자의 거주지 선관위원장 직인이 찍힌 투표용지로 인쇄해서 준다.]

사전투표는 해야겠는데 시간이 별로 없어 투표시간을 조금이라도 줄이고 싶다면 [[신분증]]은 [[주민등록증]]이나 [[운전면허증]]이 좋다. 사전투표의 경우 신분확인기 기계에 [[신분증]]을 넣어서 본인확인 절차를 자동으로 처리한다.[* 참고로 이 과정에서 신분증 [[이미지]]를 [[스캔]]하여 본 선거 종료 시까지 선관위 서버에 보관한다. 만에 하나 사전투표를 마친 유권자가 본선거일 투표소나 다른 사전투표소에 또다시 나타나 중복투표를 시도할 때 이미 투표했다는 [[증거]]자료로 남기기 위해 공직선거법 제158조 2항에 의해 규정된 것이므로 개인정보를 보관하는 데는 문제가 안 된다. 이때문에 본투표일에 사용하지도 않는 신분증 확인 기계를 투표소마다 비치해둔다. 오른손 [[엄지]]의 [[지문]]을 [[스캔]]하거나 전자펜으로 이름을 서명하는 이유도 마찬가지다.] 따라서 [[민증]]이나 [[운전면허증]]의 경우 유권자 정보가 [[OCR]]로 인식되어 신분 확인이 금방 끝나는데, [[여권]]같이 신용카드 크기가 아닌 신분증이나, [[학생증]] 같이 보편적이지 않은 신분증을 챙겨가면 스캔을 통한 개인정보 인식이 안 되므로 여권번호, 발행일, 이름, 생년월일 같은 정보를 일일이 입력하느라 본인 확인에 시간이 좀 오래 걸린다.

[[부재자 투표]] 제도가 사실상 폐지되면서 부재자 신고를 하게 되는 것은 [[선상투표]]와 [[거소투표]]에 한정된다. [[재외선거]]는 예외다.

[[대한민국]]에서는 [[대한민국 대통령 선거]], [[대한민국 국회의원 총선거]], [[전국동시지방선거]] 등 전국단위 대규모 [[선거]]에서는 제한된 거소투표([[우편]][[투표]])와 대부분의 사전투표소 투표가 진행되지만, [[재보궐선거]]에서는 선거가 실시되는 지역에만 사전투표소가 설치되므로, 선거가 실시되는 곳이 아닌 지역에 거주하는 유권자에 한해서는 거소 투표를 신청하여야 한다.

단, 이 경우에도 [[재보궐선거]]가 실시되는 지역의 사전투표소에서는 같은날 실시되는 다른 지역 재보궐선거의 사전투표도 가능하므로, 만약 현 거주지에서도 같은 날 재보궐선거가 치러진다면 거소투표신청을 하더라도 받아주지 않으며, 사전투표날에 그 지역의 사전투표소에 찾아가도록 안내한다. 예를 들어 만약 서울시교육감과 서귀포시 국회의원 보궐선거가 같은 날에 실시될 경우, 서귀포시에 있는 서울시민[* 교육감 보궐선거이므로 조건을 만족시키는 한국 영주권 취득 외국인도 유권자에 포함된다.] 은 거소투표를 신청할 필요 없이 서귀포시에 있는 아무 사전투표소에나 가서 관외선거인에 줄선 뒤 시울시교육감 투표하러 왔다고 신분증 들이밀면 확인 후 서울시교육감 투표용지를 뽑아준다. 하지만 같은 날에 제주시에서는 어떠한 재보궐선거도 없다면, 제주시에 기거하는 서울시민은 거소신고를 해야 투표할 수 있다. 아니면 신고하지 않고 사전투표일에 서귀포시로 내려와서 투표하고 가도 된다.

사전투표일은 선거일 5일 이전부터 2일간 '''06시 ~ 18시'''에 실시된다.[* [[공직선거법]] 제155조 제2항 본문에 있다.] 부재자투표 당시의 10 ~ 16시에서 확장되었다. 선거가 갑자기 생기는 [[제19대 대통령 선거|지극히 예외적인 경우]]가 아니면[* 선거일이 화요일로 결정된 관계로 사전투표일은 목/금이 되었다. 다만 이 금요일이 [[어린이날]]이었던 관계로 금/토에 치르는 것과 별반 차이가 없는 효과가 났기 때문에 이렇게 결정하였다. 즉, 선관위나 정부에서도 휴일이 하루 끼어있도록 운영한다는 사실상의 숨어있는 원칙이다.] 공직선거법상 원칙적으로 모든 투표는 수요일에 실시하므로 사전투표는 선거를 실시하기 바로 전 '''금요일, 토요일'''이 된다.

사전투표를 완료한 유권자가 해당 선거의 선거일 전에 사망하는 경우가 있을 수 있는데, 이런 경우 공직선거법 제179조(무효투표)의 4항에 규정된 무효투표의 예외사유 중 하나에 포함되어 유효표로 인정된다. 거소, 선상투표도 같다. 사실 법적인 걸 떠나서, 이미 투표함에 던져진 수많은 표 중에 어떤 게 죽은 사람 표인지 딱히 구분할 수 있는 방법도 없다.

=== 사전투표소 설치 ===
[[대한민국]] 전국 사전투표소 중 '''90% 이상의 사전투표소를 [[읍(행정구역)|읍]], [[면(행정구역)|면]], [[행정동]]의 [[행정복지센터]]에 설치'''한다. [[주민센터]]에 설치하는 이유는 가장 말단의 행정기관을 담당하고 있는데다 전국적인 [[지방(지리)|지방]] 행정 [[전산]]망을 미리 설치해두었기 때문이다.

사전투표제로 바뀌면서부터는 대학교 내에 사전투표소가 설치되는 경우가 거의 없어졌다. [[http://www.nec.go.kr/portal/bbs/view/B0000342/10950.do?menuNo=200035|선관위 브리핑(2014년)]]에 의하면, 투표용지를 발급할 수 있는 국가통신망이 없다는 점과, 인근 읍 / 면 / 동사무소를 이용할 수 있다는 점 때문에 대학교 내 사전투표소를 만들지 않는다고 한다. 한편, [[군대]]의 경우는 특별한 경우가 아니면 투표소가 영외에 마련된다. 다만 현실적으로 여건이 안되는 극히 일부 지역의 경우 영내와 영외의 어정쩡한 경계에 있는 면회실(회관) 같은 곳에 설치되는 경우는 있다. 이때도 참관인들이 [[간부|높으신 군바리]]들이 깝죽거릴까봐 눈에 불을 켜고 쳐다본다. 1992년에 일어난 [[군 부재자투표 부정 폭로 사건]] 때문에 이렇게 바뀐 것이다. 군대의 상명하복 구조의 특성상 [[높으신 분들]]이 특정 후보, 주로 투표 시점에 정권을 잡고 있었던 여당 후보를 찍도록 강요하는 일이 일어날 가능성이 높기 때문이다.

[[교통]]시설에 사전투표 시설이 부족한 것에 대해 유권자들 사이에서 불만이 무지막지하게 많다. [[공항]]의 경우 인천공항 다음으로 이용객이 많아서 흘러 넘치는 [[김해국제공항]], [[김포국제공항]]이나 여행객이 많은 [[제주국제공항]]에는 사전투표소가 없어 투표를 못 하는 사람들이 많기 때문이다. 철도역 역시 서울역, 용산역 역시 이외에 이용객이 가장 많은 철도역인 [[부산역]], [[동대구역]] 등에도 투표소가 없어 불만이 끊이질 않는다. [[http://www.kookje.co.kr/news2011/asp/newsbody.asp?code=0300&key=20170505.22003221015|부산역, 김해공항 사전투표소를 설치해달라는 기사]]

비슷하게 유동인구가 많은 시설인 [[버스 터미널]] 중에는 투표소가 설치되는 곳이 하나도 없다. 이렇기 때문에 기타 여러 시설에 사전투표소 설치 요구가 끊이질 않는데, 선관위에서는 서울역, 용산역, 인천공항 3곳도 국회에서 태클 들어온다며(국회의원들은 특별 사전투표소 확대에 대해 정파에 따라 의견이 극명하게 갈린다) 다른 교통시설에 대한 사전투표소 설치에는 부정적이다. 당장 이 세 곳도 원래는 인천공항만 [[부재자 투표]] 시절부터 투표소가 있었고 [[서울역]] 투표소도 2014년 [[제6회 전국동시지방선거]] 때, 용산역 투표소도 2016년 [[20대 총선]]에 추가된 곳이다. 일단 선관위에서 낸 교통시설 사전투표소 관련 제출의견에 보면 추가로 생기는 후보지는 서울에서는 김포공항, [[청량리역]], [[수서역]], [[영등포역]],  [[강남역]], [[서울고속버스터미널]], [[센트럴시티]], 그 외 지역에는 제주공항, [[수원역]], [[부산역]], [[동대구역]], [[대전역]], [[광주송정역]], [[유스퀘어]]까지는 포함되어 있다.

==== [[행정복지센터]] ====
[[파일:20200411222828.jpg]]

위의 사진은 [[인천광역시]] [[서구(인천광역시)|서구]] [[당하동]] 행정복지센터이다. 가장 일반적인 사전투표소로, 주로 2~3층에 있는 대회의실을 사용한다. 가장 일반적인 사전투표소이다.(단, 일부 지역의 경우 주민자치센터와 같이 행정복지센터와 비슷하지만 기능이 분산된 곳에 설치되는 경우도 제법 있다. 엘리베이터 없이 3층에 있거나 해서 노약자, 장애인 편의성이 떨어지는 경우도 있고, 시골 면사무소 같은 경우 대회의실 같은 시설이 아예 없는 경우도 있기 때문이다.)

==== 상급 [[지방관청]] 활용 ====
[[파일:201806081793349644_1.jpg]]

자치구청인 [[서울특별시]] [[종로구]]청 사전투표소. [[https://www.hankookilbo.com/News/Read/201806081793349644|기사]]

[[파일:200410095344399.jpg]]

[[일반구]]청인 [[경기도]] [[고양시]] [[덕양구]]청 사전투표소. [[http://www.newspim.com/news/view/20200410000208|기사]]

[[파일:12609_12244_1025.jpg]]

[[기초자치단체]] 청사인 [[경상남도]] [[진주시]]청 사전투표소 [[http://www.jjinews.net/news/articleView.html?idxno=12609|기사]]

드물게 주민센터가 아닌 [[구청]], [[시청(행정)|시청]], [[군청]]에 설치하기도 한다. 구청은 [[자치구]]청과 [[일반구]]청 두 군데 모두 담당할 수 있다. 구청, 시청, 군청에 사전투표소를 설치할 경우 해당 시설과 소재지가 같은 읍면동의 [[행정복지센터]]에는 사전투표소를 설치하지 않는다.

[[파일:1286370_1081580_1133.jpg]]

[[충청북도청]]에 설치한 [[청주시]] [[상당구]] 성안동 사전투표소. [[http://www.jbnews.com/news/articleView.html?idxno=1286370|기사]]

[[파일:PYH2020041003020006300_P4.jpg]]

[[대전광역시청]]에 설치한 대전광역시 [[서구(대전)|서구]] 둔산1동 사전투표소 [[https://www.yna.co.kr/view/AKR20200410054251063|기사]]

극히 드문 사례인데, '''위엄쩔게 [[광역자치단체]] 본청 청사에 사전투표소를 설치하기도 한다'''. 광역자치단체 본청 청사는 워낙 상급기관이라 사전투표소 설치가 부적합한 기관이다. 선거인명부를 작성하는 곳은 '''[[기초자치단체]]'''이기 때문이다. [[대한민국]]도 [[지방자치제도]]를 하면서 광역과 기초 간에 업무가 분담되어 있는데 [[선거]]의 경우 대표적인 '''기초자치단체 고유사무'''에 속한다. 이 때문에 선거인명부를 작성하지 않는 광역자치단체 본청에 사전투표소 설치를 위해서는 시군구청장과 시도지사가 직접 협상을 한 후에 전산망 추가작업을 해야 한다. 달리 말하면 정치력, [[시간]], [[예산]]이 따로 들어간다는 뜻이다.

==== [[도서관]], [[체육관]] 등 타 [[공공기관]] 관청 ====
[[파일:2020041101137_0.jpg]]

[[2020년]] [[제21대 국회의원 선거]] [[경기도]] [[고양시]] [[일산동구]]에 위치한 [[백석도서관]] 사전투표소(백석1동 사전투표소) 현장. [[https://biz.chosun.com/site/data/html_dir/2020/04/11/2020041101145.html|기사]]

[[파일:2020041001000396300016131.jpg]]

[[경상북도]] [[포항시]] [[북구(포항)|북구]] 양학국민체육센터에 마련된 양학동 사전투표소. [[https://www.yeongnam.com/web/view.php?key=20200410010001613|기사]]

[[파일:PYH2020041504770001300_P4.jpg]]

'''[[게이트볼]]장에 설치'''한 [[세종특별자치시]] [[소정면]] 사전투표소. [[https://www.yna.co.kr/view/PYH20200415047700013|기사]]

사전투표 제도가 시범기간([[2014년]] [[제6회 전국동시지방선거]]) 이후 [[2016년]] [[제20대 국회의원 선거]]부터 각급 [[학교]] 대신 [[지방자치단체]]가 운영하는 [[도서관]]이나 [[체육관]] 사전투표소를 설치하는 경우도 보이고 있다. 이런 곳에 사전투표소를 설치하는 것은 '''전산망 작업을 추가로 해야하는 부담이 있음에도 불구'''하고, [[주민센터]] 위치가 좋지 않거나 이런 공공기관 청사가 주민센터에 비해 '''압도적으로 접근성에 우위'''에 있다는 뜻이다. 위에 예시로 나온 백석도서관은 주변에 일산병원, [[백마고등학교]], 유니테크빌 [[지식산업센터]], [[고양종합터미널]], [[일산 와이시티]]가 위치해 있어 백석1동 주민센터에 비해 입지상 우위이다.

[[세종특별자치시]]에서 [[행정중심복합도시]]가 아니라 읍면 지역은 [[노인]]들이 대부분 거주하는데, 이들이 편리하게 투표할 수 있도록 [[행정복지센터]]가 아니라 '''[[게이트볼]]장에 사전투표소를 설치'''하는 이색 투표소를 운영한다.

==== 각급 학교 체육관, 강당 ====
이 모든 곳이 안되는 경우에는 학교 체육관이나 강당에 설치되는 경우도 있다. 대표적인 케이스가 서울 서초3동인데, 여기는 주민센터가 너무 작아 사전투표소를 만들기 어렵고[* 현재는 비좁은 구 주민센터를 자치센터로 리모델링하고, 주민센터는 아예 일반 빌딩 2층에 세들어있다. 즉, 투표소를 만들 곳이 없다.] 다른 공공기관도 주위에 없어서, 사전투표제도가 도입된 이후 항상 [[서울고등학교]] 강당에 투표소가 설치된다.

그 외에도 규모가 작은 시골 면의 경우 면사무소 건물이 작거나 1층에는 사무 공간밖에 없는 등 적합치 않아서 인근의 초등학교 강당이나 체육관, 농협 회관 , 주민회관 등을 이용하는 경우가 많다.(이 경우는 대부분 같은 이유로 본투표소 역시 같은 곳에 설치된다.)

==== [[금융]]기관 ====
[[부산광역시]] [[사하구]] 감천1동 사전투표소는 사하중앙새마을금고(구 감천1동새마을금고) 본점에 설치된다. 감천1동 [[행정복지센터]]가 [[1950년대]] 지은 노후 건물이라 맞은편의 [[새마을금고]] 건물을 활용하고 있다.

==== 대피소(지하 [[벙커]]) ====
[[2010년]] [[북한]]의 '''[[연평도 포격 도발]]'''을 겪은 [[인천광역시]] [[옹진군(인천)|옹진군]] [[연평면]]은 '''연평도 4호 대피소(지하벙커)'''에 사전투표소를 설치하는 '''위엄 쩌는 모습'''을 보인다. [[투표]]를 하는 도중에 [[북한]]에서 또 군사 도발을 하여 [[포탄]]을 날릴 수 있으니 '''투표소 자체를 지하벙커에 설치하는 것'''이다. 연평면은 [[2010년]] 연평도 포격 도발 이후로는 선거일투표소 3곳도 모두 지하벙커에 설치한다. [[중앙선거관리위원회]]의 유튜브 방송에서 벙커 투표소 사실을 언급할 때 댓글로는 "사전투표가 아닌 '''안전투표'''" 드립이 성행했다.

지하벙커에 설치하는 연평면 투표소들은 '''군사보호구역으로 되어 있어서 [[사진]] 촬영이 제한된다'''.

==== 특별 사전투표소 ====
[[파일:2017050401000143400012991.jpg]]

[[서울역]] 사전투표소([[서울특별시]] [[용산구]] [[남영동]] 사전투표소) [[http://www.asiatoday.co.kr/view.php?key=20170504010001299|기사]]

[[파일:wef24f24f24g2g4.jpg]]

[[용산역]] 사전투표소(서울특별시 용산구 [[한강로동]] 사전투표소)

[[파일:152816_101870_213.jpg]]

[[인천국제공항]] 출국장 사전투표소([[인천광역시]] [[중구(인천)|중구]] [[운서동]] 제2사전투표소) [[http://www.1gan.co.kr/news/articleView.html?idxno=152816|기사]]

특별 사전투표소로는 [[서울역]], [[용산역]], [[인천국제공항]] 제1여객터미널, [[인천국제공항]] 제2여객터미널 4곳만 지정되어 있다. 이 4곳은 다른 사전투표소들이 대부분 '''관내선거인''' 투표가 대다수인 것과 달리 '''관외사전투표'''가 절대 다수를 차지한다.

=== 개표 ===
[[파일:12121111.jpg]]

[[파일:12121112.jpg]]

사전투표 후 [[개표]]장으로 옮겨진 관내, 관외사전투표함. 관내사전투표함은 사전투표일에 투표한 이후, 관외사전투표함은 우체국으로부터 인계받은 후 분류하여, 개표일까지 구시군 선거관리위원회에 보관해야 하기 때문에 보관에 용이하게 밑이 [[검은색]] 행낭으로 이루어져 있다.  관외에서 사전투표 된것은 선관위에서 선상투표, 거소투표 표와 같이 개표한다.

[[파일:1212111346.jpg]]

[[파일:12121113480.jpg]]

[[파일:1212111347.jpg]]

관외 사전투표 투표함 개표장면. 저 어마어마한 수량의 봉투를 일일이 잘라낸다.

[[부재자 투표]] 때와 달리 사전투표는 참가자 비율이 크게 늘어났기 때문에 부재자 투표에 비해 사전투표의 [[개표]]는 시간이 상당히 걸린다고 한다. 덕분에 과거 부재자투표 시절이나 사전선거 때나 맨먼저 시작하는 건 같은데, 부재자 투표함 때는 맨먼저 개표해서 맨먼저 끝이 났던 데 비해, 사전투표제로 바뀐 이후로는 관외사전투표는 아예 따로 판을 차려서 제일 먼저 시작해도[* [[거소투표]], [[선상투표]], 관내사전, [[재외선거|국외부재자투표]], 관외사전이 먼저 [[개표]]가 개시된다. 이 중 국외부재자투표 또는 거소&선상투표가 가장 빨리 발표되고, 관외사전은 개표를 제일 먼저 시작해도 워낙 표 수가 많아 제일 늦게 끝난다.] '''제일 늦게 끝난다'''.(워낙 오래 걸려서 판을 따로 잡으므로 사전투표소와 관내투표소 중 한곳을 동시에 시작한다.) 당장 20대 총선에서 바로 여기에서 당락이 뒤집힌 케이스가 여럿이 있다.

2016년 총선에서 [[진주시]]의 사전투표 중 [[비례대표]]에서 새누리가 몰표를 얻었는데 [[http://www.idomin.com/?mod=news&act=articleView&idxno=506393|정작 몇몇 사전투표에 참여한 유권자들이 새누리를 찍은 적 없다 말하는 상황이 나오고 있다.]] 조사 결과 선관위 직원의 기재 착오로 나타났다. 두 지역의 결과를 직원이 멋대로 합산해버려서 한 지역에서는 새누리 100%, 다른 지역에서는 새누리표가 적게 나오고 야당 표가 실제보다 부풀려진 것.

2017년 대선에서 대선 최초로 사전투표가 실시되었다. [[중앙선거관리위원회]]에 따르면 4~5일 이틀간 치른 사전투표에 총 선거인 4247만 9710명 중 1107만 2310명이 투표, 투표율 26.06%를 나타냈다. 이는 2013년 사전투표제가 도입된 이후 가장 높은 투표율이었으나, [[2020년]] 4월 11일 18시 기준 [[제21대 국회의원 선거]]가 선거인 4399만 4247명 중 1174만 2677명이 투표한 26.69%를 기록하면서 0.63%p 차이로 최고 사전투표율 기록이 바뀌게 되었다.

=== 투표율 및 개표결과와의 연관성 ===
사전투표가 전체 투표율에 끼치는 영향에 대해서는 해석이 약간 엇갈리는 편이다. 일단 사전투표 자체가 전체 투표율 상승에 도움이 된다는 해석이 주류를 차지한다. 이전의 [[부재자 투표]]에 비해 절차가 간편해서 선거 당일에 투표를 하지 못하는 유권자들이 더 투표에 참여할 수 있다는 해석이다. 하지만 투표율에 도움이 되는 것 자체는 사실이나 의외로 제한적인 수준이라는 해석도 있다. 유권자들이 선거 당일이 아닌 사전투표일에 투표를 실시해서 투표율이 분산되는 경우도 있기 때문이다.

다만 전체 투표율을 떠나 개표 결과에는 어느 정도 영향을 미치는 것으로 보인다. 특히 관외 사전투표가 활성화되면서 기존의 병역뿐 아니라 직장, 학업 등의 이유로 타지에 나가 있는 사람들의 투표가 부쩍 늘었고, 이들은 주로 젊은층이 많은데다 아무래도 지역에 계속 거주한 사람들과 생각이 다를 수 있기 때문에 이쪽에서 막판에 몰표가 쏟아지면서 결과가 뒤집어지는 경우도 종종 발생한다.

예를 들어 제7대 지방선거 구미시장의 경우 구 선산군+구미시 도심과 그 외 구미시 지역의 표심이 극명하게 갈리며 자유한국당 후보가 미세하게 앞섰으나 관외 사전투표 개표에서 민주당 후보에게 몰표가 쏟아지며 사상 처음으로 민주당 후보가 당선되었다.

역대 최고 사전투표율 기록을 경신했던 [[제21대 국회의원 선거]]에서는 이런 차이가 극단적으로 드러났다. 사전투표만 계산하면 [[더불어민주당]]이 253개의 지역구 중 200곳 이상에서 1위를 한 반면, 사전투표를 제외하고 본투표 결과만으로 당선자를 산출하면 미래통합당이 원내 1당이 될 정도로 차이가 극명했다. 이는 이후 보수층 일각에서 결과를 받아들이지 못하고 [[제21대 국회의원 선거/부정선거 음모론|사전투표 조작 음모론]]을 펼치게 되는 배경이 되기도 했다.

이후 선거에서도 이런 경향은 계속 이어졌다. 더불어민주당이 참패했던 [[2021년 재보궐선거]]에서도 박영선 후보가 단 한 자치구에서도 승리하지 못했던 최종 결과와는 달리 사전투표에서는 25개 자치구중 11개 자치구와 절반 가까운 동에서 승리한것으로 알려졌다. [[https://www.yna.co.kr/view/AKR20210408103100001|#]]

[[제20대 대통령 선거]]에서는 위의 부정선거 음모론 때문에 보수층이 사전투표를 기피하는 경향이 커져서 이런 현상이 더욱 심해질 것으로 예상되고 있다. 2022년 대선을 앞두고 실시한 여론조사[* 조사의뢰자: KBS, MBC, SBS. 조사기관명: 입소스(공동조사기관명: 코리아리서치인터내셔널, 한국리서치). 조사일시: 2022년 2월 15일 10~21시, 2월 16일 10~19시. 그 밖의 사항은 [[https://nesdc.go.kr/portal/bbs/B0000005/view.do?nttId=9992|중앙선거여론조사심의위원회 홈페이지]] 참조]에서는 사전투표할 의향이라는 응답자 중에서는 [[이재명]] 후보가 52.8%, [[윤석열]] 후보가 22.2%의 지지를 받았지만, 본투표 당일 투표할 의향이라는 응답자 중에서는 윤석열 후보가 48.5%, 이재명 후보가 28.9%의 지지를 받아 완전히 정반대의 결과가 나타나기도 했다.

진보 성향 유권자가 보수 성향 유권자보다 상대적으로 사전투표에 더 많이 참여하는 것은 한국에서만의 추세가 아닌 전 세계적인 추세로, [[미국]]에서도 사전투표에서 [[민주당(미국)|민주당]]이 [[공화당(미국)|공화당]]을 압도하고 사전투표 특성상 마지막에 개표되다 보니 개표 과정 막바지에 민주당이 역전승하는 경우가 많은 편이다.
=== 장점 ===
본투표와 비교하면 편리하다는 것이 중론. 본투표에서는 통반에 따라 정해진 투표소에 가지 않으면 투표가 불가능하나, 사전투표에서는 장소와 지역에 구애받지 않고 투표가 가능하다. 또 본투표의 경우 통반으로 분배된 선거인의 명부를 가지고 있고, 이를 투표소 근무자들이 일일이 찾아야하므로 찾는 근무자도 힘들고 그만큼 대기시간이 길어진다. 사전투표로 진행하면 위의 열거된 특징으로 시간이 상당 부분 단축된다. 선거를 관리하게 되는 선거관리위원회 및 [[지방관청]](특히 동 [[주민센터]]) 역시 사전투표제의 형식을 상당히 선호한다. 장소가 구애되지 않아 동 주민센터나 공공기관 청사 등의 거점만 확보하면 되기 때문이다.

본투표에서는 적절한 장소를 찾기가 매우 어려운 상황이다. 예를 들어 일개의 동에 투표소를 골고루 배치해야 하는데, 골고루 배치하면서도 투표소의 조건을 충족할만한 공간이 나오는 장소를 확보하기가 몹시 어렵다. 시간이 갈수록 장소를 대관해주는 개인이나 집단이 점차 줄어들고 있다. 그나마 과거에는 교회의 협조를 받아 교회, 성당 등 종교시설의 공간을 활용한 적이 있었는데, 종교 편향의 시비가 나오면서 현재는 선거법에 종교시설에 투표소 설치가 금지되어 있다.[* 예를 들면 불교 신자가 교회에 설치된 투표소에서 투표를 하고 나오는데, 해당 교회의 신도가 교회 앞에서 전도를 시도하면 많이 불편할 것이다.] 그러다 보니 최근에는 하나의 학교에 다수의 투표소가 설치되는 경우도 많다.[* 예를 들면 A구 B동에 위치한 C초등학교 1학년 1반 교실은 해당 동의 제1투표소가, 같은 학교의 2학년 2반 교실은 제2투표소가 되는 식이다.]

또한 상당 부분 전산화되어 노동력을 줄여주는 데다가, 왜 이리 투표지 배부가 늦냐는 투표자들의 채근에서도 벗어날 수 있다. 또 [[투표용지]]를 적절히 배분하는 과정도 단순화 되어 배부 과정에서 생기는 실수등에서도 상당 부분 해방된다. 본투표에서는 사전에 선거인수를 고려한 투표지를 미리 지급받고, 해당 '''투표용지들은 각자 고유의 시리얼넘버가 부여되어 있다.''' 추후 당일 선거 종료 후에 선거가 정확하고 공정하게 실시되었는지를 검사하기 위해 배부된 투표지와 및 잔여 투표지의 수를 파악해야 하는데, 이렇게 준비된 투표지를 지급하는 과정에서 순서가 꼬이거나, 보관중에 손망실이 생기면, 남은 투표지와 지급된 투표지의 계산이 안 맞게되어 애로사항이 꽃핀다.

[[코로나19]] 사태가 진행중이던 [[2020년]]에 치뤄진 [[21대 총선]]에서는 조금 다른 장점이 부각되기도 하였는데, 바로 본 투표일에 사람이 많이 몰리며 우려될수 있는 전염병 감염 위험성을 사전투표를 통해 투표인을 도합 3일간 분산시킴으로써 낮출수 있다는 점이 그것이다.

최근 사전투표는 이틀 정도 시간을 주고, 이틀 중 하루는 주말을 포함하기 때문에 급한 일이 있을 수도 있는 본투표날보다 매우 편리하다. 사실상 투표를 사흘 동안 하게 되는 효과가 생겼다. 특히 본투표일이 평일이라 직장인들같이 출근을 해야하는 날에 미리 투표할 수 있다.
=== 문제점 및 논란 ===
투표율 상승에 도움이 된다는 장점이 있지만 몇가지 문제점도 존재한다.

 * '''관외 사전투표함 CCTV 미설치''': 관내 사전투표함은 CCTV로 보관하는 동안 계속 감시가 되는 반면, 관외 사전투표함 보관장소에는 CCTV를 설치하지 않기 때문에 [[부정선거]] 가능성에 대한 논란이 있다. 특히 보관장소에 CCTV를 설치해달라는 민원에 중앙선관위가 거부의사를 밝힌 일로 인해 논란이 더 커진 바 있는데, 이에 대한 2019년 10월 8일 국정감사에서 선관위는 "투표용지가 하루에도 두세 번씩 배달되는데, 여러차례 투표함을 여닫는 모습이 CCTV에 나올 경우 또 다른 오해를 낳을 수 있기 때문" 이란 답변을 내놓은 바 있다.
  * 하지만 여타 [[음모론]]이 다 그렇듯, 실제로 알고 보면 사전투표함의 보안과정은 CCTV 하나에 맡길 정도로 허술하지 않다. 투표함의 밀봉시에는 각 정당에서 신청한 선거참관인들과 선거관리인의 서명이 들어간 특수봉인지[* 단 한번이라도 떼진 흔적이 있으면 "'''OPEN VOID'''"라는 숨겨진 글귀가 무늬로 드러나 훼손 사실을 알린다.]로 밀봉된 다음 정복 경찰공무원과 참관인들의 감시 하에 관할 선거관리위원회로 이송된다. 게다가, 우편으로 온 "관외"사전투표 투표지를 투표함에 넣을 땐 '''각 정당에서 추천한 위원들이 참관'''한다. 정당 참관위원들이 언제든지 이의를 제기할 수 있고, 이송 단계에선 경찰공무원이 동행하므로, '''보는 눈이 많아서 안전'''하다. [[http://www.kookje.co.kr/news2011/asp/newsbody.asp?code=0100&key=20200407.33002002328|#]][[http://news.tvchosun.com/site/data/html_dir/2020/04/11/2020041190047.html|#]] 선거 개표 당일까지 나머지 긴 시간 동안 CCTV가 없는 선관위 사무국(과)장실에 출입하는 사람은 '사무국(과)장, 선관위 직원들'이 전부이지만, 투표함에 넣어진 투표지는 관내이든 관외이든 특수봉인지를 통해 밀봉되므로 사실상 이런 의혹도 그냥 의혹 수준에서 그칠 뿐이다. 선거때마다 하찮은 거짓증거를 들이대며 음모론을 주장하는 자들이 꼭 있지만, 팩트체크를 해보면 결국 매번 거짓으로 드러날 뿐이다.
  * 이런 보안에 대한 우려를 제기하는것은 만약의 여지를 없앰으로써 선거의 공정성을 더욱 확실시 하기 위해서는 당연한 처사임이 분명하고, 이에 대해 이견은 그리 없을것이다. 하지만, 이것에서 더 나아가서 실제로 부정선거가 있다고 주장하며 끊임없이 [[음모론]]을 제기하는 사람들이 적지만 있긴 하다. 그들의 주장에는 크나큰 맹점이 있는데, '''선거조작의 주체가 있다고 미리 결정지으며 그 주체를 전지전능하다고 믿어버리는 것'''이다. 애초에 선거조작을 하면서 이득을 보려는 정당이 있다면, 다른 정당들이 '''죽일듯이 추적'''해서 낱낱이 다 파내버릴 것이 확실한데, 민주주의가 공고해진 한국에서는 이렇게 들킬 때의 정치적 리스크가 너무 크다.

 * '''사전투표 후~본투표 전 이슈 미반영''': 어느 선거든 선거 막바지 쯤에 이슈가 터질 가능성[* 2002년 [[제16대 대통령 선거]]에서 '''투표일 전날 밤''' [[정몽준]]이 [[노무현]] 후보와의 단일화를 파기한 것이 대표적이다.]이 충분히 존재하는데 이 경우 사전투표에서 미리 투표한 유권자들은 그러한 이슈에 대한 자신의 판단과 의사를 표시할 수 없게 된다. 사전투표와 본투표 간의 시간 간격을 줄이면 이럴 확률을 줄일 수는 있지만, 완벽히 방지하기란 불가능하다. 때문에 보통 사전투표에 적극적으로 참여하는 유권자는 어지간한 이슈가 터져도 지지 후보나 정당을 바꿀 의향이 없는 [[콘크리트 지지층]]들이 많다.

 * '''더욱 짧아진 공식 선거운동기간''': 대부분의 서유럽 및 북아메리카 국가들과 달리, 대한민국은 공식 선거운동기간을 법으로 지정하고 있다. 공식 선거운동기간 이전에는 다양한 방식의 홍보수단이 금지되어 있는데, 이러한 제도는 인지도가 낮은 군소정당이나 무소속 후보에게 특히 불리하다는 지적이 있어왔다. 그래서 공식 선거운동기간을 아예 없애거나 현행법상 대통령 선거의 경우 23일, 국회의원 및 지방자치단체 선거의 경우 13일로 지정된 공식 선거운동기간을 늘리자는 의견이 끊임없이 제기되었다. 하지만 사전투표제도는 가뜩이나 짧은 공식 선거운동기간을 실질적으로 4-5일씩 더 줄여버리는 효과를 가져온다.

 * '''여론조사 공표금지 기간 무력화''': [[공직선거법]] 제108조 1항[* 제108조(여론조사의 결과공표금지 등) ① 누구든지 선거일 전 6일부터 선거일의 투표마감시각까지 선거에 관하여 정당에 대한 지지도나 당선인을 예상하게 하는 여론조사(모의투표나 인기투표에 의한 경우를 포함한다. 이하 이 조에서 같다)의 경위와 그 결과를 공표하거나 인용하여 보도할 수 없다.]은 선거 6일 전부터 여론조사의 공표를 금지하고 있는데, 사전투표가 포함되어 있으면 사실상 공표금지 기간이 1~2일로 짧아져 법이 무력화되는 효과가 발생한다.

 * '''읍면동별·투표소별 개표 결과 왜곡''': 관외사전투표 투표지들은 해당 유권자의 거주지역 선관위로 옮겨진 뒤 개표되는데 이때 관외사전투표의 경우 시군구 단위로 별도개표 및 집계되기 때문에 읍면동 단위에서의 개표결과 통계에 왜곡이 발생되게 된다. 사전투표 투표자들은 주로 젊은층들이 많고 이에 따라 관외사전투표 개표결과에서는 진보정당 후보 득표율이 상대적으로 높고, 대신 나머지 읍면동 단위의 개표결과에서는 보수정당 후보 득표율이 상대적으로 높아지는 현상이 나타나게 된다. 시군구 단위만큼이나 읍면동 단위의 개표결과도 정치학적으로 중요한 연구자료이자 거주민들의 정치적 의사와 판단의 표출수단인 만큼 이는 사전투표의 문제점 중 하나로 볼 수 있다. 한국 선관위가 읍면동 단위로 관외사전투표 단위를 개편하면 되는 일이지만 매우 귀찮은 작업이 동반되기 때문에 그냥 시군구 단위로 유지하고 있다.
 사실 관내사전투표의 경우에도 통계가 읍면동으로 이뤄지고는 있으나, 이는 그 해당 읍면동에서 투표한 사람이 대상이지 읍면동에 거주하는 사람이 아니라는 점에서 정확한 통계자료는 아니다. 예를 들어 단일지역구인 도농통합시의 A면에 거주하는 사람이 일보러 시내에 나왔다가 B동에서 사전투표할 경우 그 투표용지는 B동의 관내사전투표함에 들어 있으므로 이 사람의 투표 결과도 A면이 아니라 B동의 관내사전투표자에 반영된다. 따라서 접근성이 높은 읍면동의 관내사전투표는 과대표집되는 통계의 왜곡이 발생한다.
 그리고 사전투표는 투표소별 개표 결과 반영이 불가능하다는 단점도 가지고 있다. 하나의 읍면동 안에서도 투표소별로 개표 결과가 큰 차이를 보이는 경우가 많은데,[* 특히 읍면동 내에서도 각 투표소들이 속한 지역 간 경제력 격차가 심한 경우, 또는 도농복합 읍면동인 경우 이려한 경우가 많다.] 관외사전투표는 물론 관내사전투표 역시 투표소별 개표 결과를 왜곡시킨다.
  * [[대한민국]]에서 시군구 선관위는 별도의 건물을 쓸 정도로 상주 기관이다. 하지만 읍면동 선관위는 선관위의 인력 부족으로 읍면동 [[주민센터]] 내에 선관위 직원 1~2명을 그것도 순환 감독하는 식으로 운영하고, 선거 때에나 기존 [[지방자치단체]] 소속 공무원을 임시로 읍면동 선관위원으로 임명하는 식이기 때문에 해결이 어려운 것이다. 관내 사전선거인은 당연히 읍면동 단위로 집계한다. 참고로, 선관위는 [[http://www.nec.go.kr/portal/bbs/view/B0000236/24641.do?menuNo=200180|정치관계법 개정의견(2016. 8. 25. 제출)]]을 통해, 지방선거의 경우, 관외사전투표뿐만 아니라, 관내사전투표 개표도 기초의원 선거구 단위로 하여, 현행 읍면동 단위보다 넓혀야 한다고 주장한 바 있다. 물론 입법은 국회 몫이므로, 어디까지나 의견일 뿐이다.

 * '''기술적인 문제''': 투표용지를 일일이 출력을 해야되는데 이 속도가 느려서, 지방선거의 경우 7~8장을 다 뽑으려면 한참 걸려서 대기 시간이 오래 걸린다. 사전투표소 설치도 애로사항이 있기는 마찬가지다. 사전투표가 진행되는 날에 일상 업무가 진행되는 평일이 끼어있기 때문이다. 주민센터 등에 회의실과 같이 넓으면서도 당장 일상 업무에는 지장없는 공간이 있으면야 사전투표가 훨씬 편하지만, 그런데가 없거나, 있다 하더라도 장애인이나 어르신들이 이동하기 불편한 구조라면 대체할 장소를 구하기가 쉽지만은 않다. 특히 투표소의 꽃(?)이라 할 수 있는 학교가 사실상 거의 불가능하고, 된다 쳐도 정상 수업 중에 투표가 이뤄지므로 아무래도 학생들의 학습권에 침해가 발생한다는 게 가장 큰 문제.[* 다만, 2020년에는 코로나19 사태로 인해 온라인 개학을 하게 된 상황이라 가능한 곳도 있을 수 있다. 강릉시 내곡동의 경우  명주초등학교에서 사전투표가 진행됐다.]
 특히 시골 면사무소는 빈 공간이 없는 경우가 많아서 결국 인근 초, 중학교에 설치되는 경우가 많은데 여기도 덩달아 규모가 작으므로 이때 문제가 좀 된다. 이런 지역은 사실 지역에서 사람들이 모이는 [[버스 터미널]]이나 [[철도역]]을 활용하면 되는데 위에서도 언급했듯이 이 역시 쉽지 않다. 사실 상당수의 역과 터미널은 유휴공간이 없거나 동선이 불편하다는 게 가장 큰 문제다.

 * '''출구조사의 정확성 저하''': 사전투표 자체의 단점이라기보다는 [[공직선거법]]으로 인해 생긴 문제점에 가깝다. 현 공직선거법 167조[* 공직선거법 제167조(투표의 비밀보장) ② '선거인은 투표한 후보자의 성명이나 정당명을 누구에게도 또한 어떠한 경우에도 진술할 의무가 없으며, 누구든지 선거일의 투표마감시각까지 이를 질문하거나 그 진술을 요구할 수 없다. 다만, 텔레비전방송국·라디오방송국·「신문 등의 진흥에 관한 법률」 제2조제1호가목 및 나목에 따른 일간신문사가 선거의 결과를 예상하기 위하여 선거일에 투표소로부터 50미터 밖에서 투표의 비밀 이 침해되지 않는 방법으로 질문하는 경우에는 그러하지 아니하며 이 경우 투표마감시각까지 그 경위와 결과를 공표할 수 없다.' ]에 의거해 본투표일 외에는 출구조사를 할 수 없다. 이 때문에 사전투표의 투표율이 높을 수록 본투표 유권자의 여론만을 취합하는 출구조사의 정확성이 떨어질 수 있다. 실제로 제21대 총선에서는 이 사전투표의 결과가 반영되지 않아서 출구조사에서 각 당의 의석수, 그리고 격전지의 승패 여부를 많이 놓치게 되었다. 선거결과를 빨리 알 수 없어서 불편한것 뿐만 아니라 출구조사로만 알 수 있는 정보들, 예를 들어 성별, 세대별 득표율 등을 정확히 파악할 수 없게 된다. 사전투표가 끝난 후 사전투표에 응한 연령대를 선관위 통계를 통해 확인할 수 있기 때문에 이를 바탕으로 통계치 보정을 하긴 하지만 사전투표와 본투표에 참여하는 유권자들의 특성이 각각 다르기 때문에 그러한 보정만으로는 한계가 있다. 다만 이는 사전투표에서도 출구조사를 허용하되, 결과의 공개는 본투표 종료후에 같이 실시하는 식으로 공직선거법을 개정하여 해결할 수 있을 것이다.

== 해외 ==
=== [[캐나다]] ===
캐나다에서는 사전투표를  advance polling이라고 한다.

=== [[유럽연합]] ===
[[유럽연합]] 각국의 [[선거]]에서도 사전투표제가 적용된다.

=== [[미국]]의 조기투표 ===
[[2000년]] [[미국 대통령 선거]]를 앞두고 몇몇 [[주(행정구역)|주]]에서 도입한 이후 [[2012년]] [[미국 대통령 선거]] 기준 34개 [[주(행정구역)|주]]와 [[워싱턴 D.C.]]에서 채택하고 있다. 보통 본 선거일 50일 전부터 본 선거일 전날까지 운영되며, 유권자의 요청이 [[지방자치단체]]로 접수될 경우 조기투표소가 설치된다. 심지어 우주에도 설치되는데, 우주투표제를 도입한 사람이 바로 미국 43대 대통령이었던 [[조지 W. 부시]]이다

==== [[2020년 미국 대통령 선거]] ====
[youtube(pDRYVgVky0A)]
[[2020년 미국 대통령 선거]]를 앞두고 미국 내 [[코로나 19]]가 장기화 되면서 본 투표일에 사람이 많이 몰릴 것을 우려해 우편투표나 조기투표를 하려는 미국 유권자들이 대폭 늘어날 것으로 보인다. 최근 조기투표를 개시한 주들의 상황을 보면 예상보다 투표 열기가 더욱더 뜨겁다고 한다. 어느정도냐면 조지아 주에서는 11시간 기다려서 투표했다고... 이것은 [[2016년 미국 대통령 선거]]의 약 7배 정도 된다고 한다, 현 추세라면 미국 역사상 최초로 대선 투표에 나서는 이들의 과반이 선거 당일 전에 한표를 행사하게 될 전망이라고 미국 워싱턴포스트가 분석했다. 조기투표에 나선 유권자들은 대다수가 미국 민주당 지지자일 것으로 분석된다.[[https://news.v.daum.net/v/20201015120218565|#]]

미국 대선의 투표 방법은 우편투표, 조기 현장 투표, 선거 당일 현장 투표로 나뉜다. 사전 투표는 우편투표와 조기 현장투표를 포괄하는 개념이다. 여기서 우편투표는 봉투를 뜯고 용지를 펴는 등 개표 시간이 꽤 걸리고 트럼프 대통령이 우편투표는 사기라는 주장도 하는 만큼 되도록이면 조기 현장투표로 몰리는 분위기이다.[[https://news.v.daum.net/v/20201018082401562|#]]

선거 분석 사이트 미국선거프로젝트에 따르면 11월 3일(현지시간) 오전 1시 기준 이번 선거 사전 투표자 수는 9965만7000여명으로 집계되었다. 거의 1억명에 가까운 미국 국민이 사전 투표에 나선 것이다. 투표 유형별로는 직접 투표자가 3572만800여명, 우편 투표자가 6393만6200여명이다. 지역별로는 캘리포니아에서 1209만500여명이 사전 투표를 마쳐 가장 열기가 높았다. 이미 2016년 대선과 비교하면 당시 현장 투표자를 합한 전체 투표자 수의 72.3%가 투표를 마친 상황이다. 정당별로는 처음엔 민주당 지지자 비율이 60% 가까이 될 정도로 압도적이었지만, 시간이 지날수록 공화당 지지자 비율도 증가해 지금은 민주당:공화당:무당파=5:3:2 수준으로 격차가 줄어든 상황이다. [[https://news.v.daum.net/v/20201103174831382|#]]

=== [[일본]]의 기일전투표 ===
일본 전역에 선거일 6일 전부터 선거일 2일 전까지 조기투표소가 설치되며, 기일전 투표 의사가 있는 사람은 '''같은 [[도도부현]] 내의 투표소에서 아무데나 투표'''할 수 있다. 단, 외지인이 많은 [[도쿄도]], [[홋카이도]], [[오사카부]], [[교토부]], [[오키나와현]]의 투표소는 일본의 전국 모든 유권자가 투표할 수 있다. [[2019년]]부터는 [[니가타현]]도 전국 투표소로 추가되는 듯 하다.

== 관련 문서 ==
 * [[부재자 투표]]
[[분류:선거]]"""

In [5]:
import re

In [19]:
macro_map(namumark_cont2)

TAG3:::from 1 \[([A-Za-z가-힣]+)
TAG3:::from 93 \[([A-Za-z가-힣]+)
98 header
TAG3:::from 107 {{{([+\-][1-5]) 
TAG3:::from 138 \[\[
TAG3:::from 181 \[\[
TAG3:::from 194 \[\[
TAG3:::from 203 \[\[
TAG3:::from 230 \[\[
TAG3:::from 238 \[\[
259 header
TAG3:::from 274 \[\[
TAG3:::from 332 \[\[
TAG3:::from 345 \[\[
TAG3:::from 363 \[\[
TAG3:::from 407 \[\[
TAG3:::from 420 \[([A-Za-z가-힣]+)
TAG3:::from 442 \[\[
TAG3:::from 459 \[\[
TAG3:::from 481 \[\[
TAG3:::from 489 \[([A-Za-z가-힣]+)
TAG3:::from 511 \[\[
TAG3:::from 528 \[\[
TAG3:::from 560 \[\[
571 header
TAG:::from 582 \n\|[^\s]
TAG2:::from 584 '''
TAG3:::from 587 \[\[
TAG2:::from 600 '''
TAG3:::from 630 \[\[
TAG2:::from 997 '''
TAG3:::from 1021 \[\[
TAG3:::from 1032 \[\[
1088 header
TAG3:::from 1099 \[\[
TAG3:::from 1214 \[\[
TAG3:::from 1229 \[\[
TAG3:::from 1259 \[\*
TAG3:::from 1272 \[\[
TAG3:::from 1287 \[\[
TAG3:::from 1316 \[\[
TAG3:::from 1374 \[\[
TAG3:::from 1389 \[\[
TAG3:::from 1404 \[\[
TAG3:::from 1433 \[\[
TAG3:::from 1495 \[\*
TA

TAG3:::from 22460 \[\[
22511 header
TAG2:::from 22593 '''
TAG3:::from 22599 \[\[
TAG3:::from 22646 \[\[
TAG3:::from 22655 \[\[
TAG3:::from 22665 \[\[
TAG3:::from 22675 \[\[
TAG3:::from 22684 \[\[
TAG3:::from 22725 \[\[
TAG3:::from 22738 \[\[
22768 header
TAG:::from 22780 \n\u0020+(\*|1\.|A\.|I\.|i\.)
TAG3:::from 22783 \[\[
TAG3:::from 22794 \[\[


{'position': [[1, 92],
  [93, 97],
  [98, 107],
  [107, 136],
  [138, 144],
  [181, 187],
  [194, 200],
  [203, 212],
  [230, 236],
  [238, 246],
  [259, 274],
  [274, 318],
  [332, 343],
  [345, 357],
  [363, 391],
  [407, 414],
  [420, 442],
  [442, 455],
  [459, 475],
  [481, 487],
  [489, 511],
  [511, 524],
  [528, 544],
  [560, 568],
  [571, 582],
  [587, 596],
  [584, 599],
  [600, 622],
  [630, 641],
  [997, 1014],
  [1021, 1029],
  [1032, 1040],
  [582, 1087],
  [1088, 1099],
  [1099, 1213],
  [1214, 1223],
  [1229, 1248],
  [1272, 1281],
  [1287, 1303],
  [1259, 1311],
  [1316, 1326],
  [1374, 1387],
  [1389, 1402],
  [1404, 1417],
  [1433, 1440],
  [1495, 1556],
  [1619, 1627],
  [1717, 1855],
  [2025, 2040],
  [2042, 2058],
  [2245, 2256],
  [2386, 2613],
  [2807, 3043],
  [3087, 3094],
  [3096, 3105],
  [3108, 3117],
  [3142, 3149],
  [3192, 3199],
  [3201, 3207],
  [3305, 3311],
  [3422, 3428],
  [3430, 3436],
  [3438, 3444],
  [3174, 3474],
  [3479, 3485],
  [3488, 3497]

In [7]:
re.match( r"\[([A-Za-z가-힣]+)", namumark_cont2[1:])

<re.Match object; span=(0, 8), match='[include'>

'include(틀:한시적 넘겨주기, other1=대한민국 제20대 대통령 선거의 사전투표 진행 상황, rd1=제20대 대통령 선거, anchor1=사전 투표율)]\n[목차]\n== 개요 ==\n{{{+1 事前投票 / Early ballot }}}\n\n[[선거]] 당일 투표가 어려운 유권자가 별다른 신고 없이 본 선거일 이전에 [[투표]]하는 제도. [[미국]]에서 [[2000년]] 조기투표 제도로 도입하고 나서 [[일본]], [[대한민국]] 등에도 도입되었다.\n\n== [[대한민국]] ==\n[[파일:1332325_article_99_20150424142312.jpg]]\n\n관외 사전투표를 하는 [[대한민국 육군]]의 [[병(군인)|병사]] 유권자\n\n[[파일:442824_233424_055.jpg]]\n\n관내 사전투표를 하는 한 [[장애인]] 유권자\n\n[youtube(6xnzoKxQg_A)][[중앙선거관리위원회]] 제작 [[제20대 국회의원 선거]] 사전투표 [[광고]]\n\n[youtube(1OGPe1NM6sw)][[중앙선거관리위원회]] 제작 [[제20대 국회의원 선거]] 사전투표 홍보영상 (출연자 [[에이핑크]])\n\n=== 법령 ===\n||\'\'\'[[공직선거법]]\'\'\'\n\'\'\'제148조(사전투표소의 설치)\'\'\'\n① 구·시·군[[선거관리위원회]]는 선거일 전 5일부터 2일 동안(이하 "사전투표기간"이라 한다) 관할구역(선거구가 해당 구·시·군의 관할구역보다 작은 경우에는 해당 선거구를 말한다)의 읍·면·동마다 1개소씩 사전투표소를 설치·운영하여야 한다. 다만, 읍·면·동 관할구역에 군부대 밀집지역 등이 있는 경우에는 해당 지역에 사전투표소를 추가로 설치·운영할 수 있다.\n② 구·시·군선거관리위원회는 제1항에 따라 사전투표소를 설치할 때에는 선거일 전 9일까지 그 명칭·소재지 및 설치·운영기간을 공고하고, 선거사무장 또는 선거연락소장에게 이를 통지하여야 하며, 관할구역 안의 투표구마다 5개소에 공고문을 첩부하여야 한

In [22]:
re.match(r"([가-힣A-Za-z]+){4,}", "가나가나가나")

<re.Match object; span=(0, 6), match='가나가나가나'>

In [24]:
macros_position = [[1,15], [2,6], [7,9], [16,30]]

In [26]:
new_macros = []
for ix, val in enumerate(macros_position):
    inclusion = False
    for x in new_macros:
        if val[0] >= x[0] and val[1] <x[1]:
            inclusion = True
    if not inclusion:
        new_macros.append([val[0], val[1]])

In [27]:
new_macros

[[1, 15], [16, 30]]